In [ ]:
##loading the whole conversation as a string below

conversation = """
Recovery Agent (RA): नमस्ते श्री कुमार, मैं एक्स वाई जेड फाइनेंस से बोल रहा हूं। आपके लोन के बारे में बात करनी थी।

Borrower (B): हां, बोलिए। क्या बात है?

RA: सर, आपका पिछले महीने का EMI अभी तक नहीं आया है। क्या कोई समस्या है?

B: हां, थोड़ी दिक्कत है। मेरी नौकरी चली गई है और मैं नया काम ढूंढ रहा हूं।

RA: ओह, यह तो बुरा हुआ। लेकिन सर, आपको समझना होगा कि लोन का भुगतान समय पर करना बहुत जरूरी है।

B: मैं समझता हूं, लेकिन अभी मेरे पास पैसे नहीं हैं। क्या कुछ समय मिल सकता है?

RA: हम समझते हैं आपकी स्थिति। क्या आप अगले हफ्ते तक कुछ भुगतान कर सकते हैं?

B: मैं कोशिश करूंगा, लेकिन पूरा EMI नहीं दे पाऊंगा। क्या आधा भुगतान चलेगा?

RA: ठीक है, आधा भुगतान अगले हफ्ते तक कर दीजिए। बाकी का क्या प्लान है आपका?

B: मुझे उम्मीद है कि अगले महीने तक मुझे नया काम मिल जाएगा। तब मैं बाकी बकाया चुका दूंगा।

RA: ठीक है। तो हम ऐसा करते हैं - आप अगले हफ्ते तक आधा EMI जमा कर दीजिए, और अगले महीने के 15 तारीख तक बाकी का भुगतान कर दीजिए। क्या यह आपको स्वीकार है?

B: हां, यह ठीक रहेगा। मैं इस प्लान का पालन करने की पूरी कोशिश करूंगा।

RA: बहुत अच्छा। मैं आपको एक SMS भेज रहा हूं जिसमें भुगतान की डिटेल्स होंगी। कृपया इसका पालन करें और समय पर भुगतान करें।

B: ठीक है, धन्यवाद आपके समझने के लिए।

RA: आपका स्वागत है। अगर कोई और सवाल हो तो मुझे बताइएगा। अलविदा।

B: अलविदा।
"""




In [ ]:
# Load model directly from hugging face
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Telugu-LLM-Labs/Indic-gemma-2b-finetuned-sft-Navarasa-2.0")
model = AutoModelForCausalLM.from_pretrained("Telugu-LLM-Labs/Indic-gemma-2b-finetuned-sft-Navarasa-2.0")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/134M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
# switching model to gpu
model.to("cuda")

# Define the input text format with placeholders

input_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
def summarize_conversation(conversation):
  input_text = input_prompt.format(
        "Summarize the following conersation in english.", # instruction
        conversation, # input
        "", # output - leave this blank for generation!
    )

  inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
  response = tokenizer.batch_decode(outputs)[0]

  # Extracting the content under "### Response:"
  start_marker = "### Response:"
  end_marker = "<eos>"

  # Find the start and end indices
  start_index = response.find(start_marker) + len(start_marker)
  end_index = response.find(end_marker, start_index)

  # Extract the response
  summary = response[start_index:end_index].strip()

  # Print the extracted response
  return summary




In [ ]:
def key_steps(conversation):

    # Construct the input text with placeholders filled
    input_text = input_prompt.format(
        "Provide next steps identified from the conversation",  # Instruction
        conversation,  # Input conversation to identify next steps
        "",  # Output placeholder (leave blank for generation)
    )

    # Tokenize the input text and prepare it for model input
    inputs = tokenizer([input_text], return_tensors="pt").to("cuda")

    # Generate a response using the model
    outputs = model.generate(**inputs, max_new_tokens=300, use_cache=True)
    generated_text = tokenizer.batch_decode(outputs)[0]

    # Extract the content under "### Output:"
    start_marker = "### Response:"
    end_marker = "<eos>"

    # Find the start and end indices of the output text
    start_index = generated_text.find(start_marker) + len(start_marker)
    end_index = generated_text.find(end_marker, start_index)

    # Extract the identified key steps from the response
    key_steps = generated_text[start_index:end_index].strip()

    return key_steps


In [ ]:
def sentiment_analysis(conversation):

    # Construct the input text with placeholders filled
    input_text = input_prompt.format(
        "Discuss the tone of each participant in the conversation and how it evolves. Use specific examples from the text to illustrate changes in their tone.",  # Instruction
        conversation,  # Input conversation for sentiment analysis
        "",  # Output placeholder (leave blank for generation)
    )

    # Tokenize the input text and prepare it for model input
    inputs = tokenizer([input_text], return_tensors="pt").to("cuda")

    # Generate a response using the model
    outputs = model.generate(**inputs, max_new_tokens=300, use_cache=True)
    generated_text = tokenizer.batch_decode(outputs)[0]

    # Extract the content under "### Output:"
    start_marker = "### Response:"
    end_marker = "<eos>"

    # Find the start and end indices of the output text
    start_index = generated_text.find(start_marker) + len(start_marker)
    end_index = generated_text.find(end_marker, start_index)

    # Extract the sentiment analysis from the response
    sentiment_analysis_result = generated_text[start_index:end_index].strip()

    return sentiment_analysis_result

In [ ]:
summary = summarize_conversation(conversation)
print(summary)

The conversation is about a loan that the borrower is having trouble paying back. The recovery agent suggests that the borrower make partial payments for the next week and then pay the remaining balance by the 15th of the following month.


In [ ]:
steps = key_steps(conversation)
print(steps)

Next steps:
1. RA ने B को अगले हफ्ते तक आधा EMI जमा करने का प्रस्ताव दिया।
2. B ने RA को बताया कि वह अगले महीने तक नया काम ढूंढ रहा है और इसलिए वह बाकी बकाया भुगतान करने के लिए कोशिश कर रहा है।
3. RA ने B को बताया कि वह अगले हफ्ते तक आधा EMI जमा कर सकता है और अगले महीने के 15 तारीख तक बाकी का भुगतान कर सकता है।
4. B ने RA को धन्यवाद दिया और अगले हफ्ते तक भुगतान करने की कोशिश की।


In [ ]:
sentiments = sentiment_analysis(conversation)
print(sentiments)

The tone of the conversation between the Recovery Agent (RA) and the Borrower (B) is empathetic and understanding. The RA is trying to help the B by explaining the importance of timely EMI payments and the consequences of not doing so. The B is also trying to be understanding and is willing to make the necessary payments to avoid further consequences. The tone of the conversation is supportive and helpful.
